In [1]:
#from veneer.manage import create_command_line

#import veneer-py

from veneer.manage import start, create_command_line, kill_all_now
import veneer
import geopandas

In [2]:
veneer_install = 'D:/PythonScriptingForGBR/VeneerPlugin/v411'
source_version = '4.1.1'
cmd_directory = 'C:\\tempveneer\\veneer_cmd'
path = create_command_line(veneer_install,source_version,dest=cmd_directory)
path

'C:\\tempveneer\\veneer_cmd\\FlowMatters.Source.VeneerCmd.exe'

In [3]:
catchment_project='D:\\Xike\\bu\\bu_reefhydro_bo_411_noMP.rsproj'
num_copies=1    # Important - set this to be a number ~ the number of CPU cores in your system!
first_port=9950

In [4]:
processes,ports = start(catchment_project,
                        n_instances=num_copies,
                        ports=first_port,
                        debug=True,
                        veneer_exe=path,
                        remote=False)

Starting C:\tempveneer\veneer_cmd\FlowMatters.Source.VeneerCmd.exe -p 9950 -s  "D:\Xike\bu\bu_reefhydro_bo_411_noMP.rsproj"
[0] Loading plugins

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\DERMTools.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\PestFileProductionTools.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\RebuildInternalSubcatchmentRaster.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\ReefHydroCalModels.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\SneakyLinkModelEditor.dll

[0] Plugins loaded (5/5)

[0] Opening project file: D:\Xike\bu\bu_reefhydro_bo_411_noMP.rsproj

ERROR[0] log4net:ERROR Failed to find configuration section 'log4net' in the application's .config file. Check your .config file for the <log4net> and <configSections> elements. The configuration section should look like: <section name="log4net" type="log4net.Config.Log4NetConfigurationSectionHandler,log4net" />

[0] L

In [5]:
vs = [veneer.Veneer(port=p) for p in ports]

In [6]:
#kill_all_now(processes)

In [7]:
#%matplotlib inline
v = vs[0]
#v.network().as_dataframe().plot()

In [8]:
sDate = v.model.get('scenario.CurrentConfiguration.StartDate')
print(sDate)

eDate = v.model.get('scenario.CurrentConfiguration.EndDate')
print(eDate)

#v.run_model()

1/01/2000 12:00:00 AM
31/03/2000 11:00:00 PM


In [31]:
the_network = v.network()
outlets = the_network.outlet_nodes()
outlets

[{'geometry': {'coordinates': [1589773.63051, -2283475.4690071], 'type': 'Point'}, 'id': '/network/nodes/258', 'type': 'Feature', 'properties': {'feature_type': 'node', 'name': 'Node on catchment SC #113', 'icon': '/resources/ConfluenceNodeModel', 'schematic_location': [0, 0]}}]

In [106]:
def processLinks(the_network, gaugeList, linkregs, currentLink, currentReg):
    
    theLinkName = currentLink['properties']['name']
    
    ## check if this link is a reporting point
    if theLinkName in gaugeList:
        currentReg = theLinkName
        print("Found this (link) network element: " + theLinkName)
    
    ## update the dictionary
    linkregs[theLinkName] = currentReg
    
    #print("Link info: " + str(currentLink))
    
    fNodeID = currentLink['properties']['from_node']
    
    #print("Node info: " + fNodeID)
    
    fNode = the_network['features'].find_by_id(fNodeID)
    #print(fNode)
    
    fNodeName = fNode[0]['properties']['name']
    #print(fNodeName)
    
    ## check if this link is a reporting point
    if fNodeName in gaugeList:
        currentReg = fNodeName
        print("Found this (node) network element: " + fNodeName)
    
    ## recurse
    #print(the_network.upstream_links(fNode[0]))
    for upLink in the_network.upstream_links(fNode[0]):
        #print("Now processing: " + upLink['properties']['name'])
        processLinks(the_network, gaugeList, linkregs, upLink, currentReg)

    return

In [107]:
noRegStr = 'NoReg'
gaugeList = ['120219A','120206A','link for catchment SC #139']

# will assign regions to links, pair with subcatchs late
all_links = the_network['features'].find_by_feature_type('link')
linkregs = {}

## Populate the dictionary of relationships first
for link in all_links:
    linkregs[link['properties']['name']] = noRegStr

## now process the network
for outNode in outlets:
    curReg = noRegStr
    
    #print(outNode.keys())
    #print(outNode['properties'].keys())
    #print(outNode['properties']['name'])
    
    thisNodeName = outNode['properties']['name']
    
    if thisNodeName in gaugeList:
        print("Found this network element: " + thisNodeName)
        ##Set the curReg
        curReg = thisNodeName
    #else:
    #    print (thisNodeName + " was not in the list")
    
    
    ## now we begin processing
    for upLink in the_network.upstream_links(outNode):
        processLinks(the_network, gaugeList, linkregs, upLink, curReg)
    

Found this (node) network element: 120219A
Found this (node) network element: 120206A
Found this (link) network element: link for catchment SC #139


In [111]:
linkregs

#help(v.network)
#help(run_results2)

{'120213A': 'link for catchment SC #139',
 'Link #8': 'NoReg',
 'Link #9': 'NoReg',
 'link for catchment SC #114': 'NoReg',
 'link for catchment SC #115': 'NoReg',
 'link for catchment SC #116': 'NoReg',
 'link for catchment SC #117': 'NoReg',
 'link for catchment SC #118': 'NoReg',
 'link for catchment SC #119': 'NoReg',
 'link for catchment SC #120': 'NoReg',
 'link for catchment SC #121': 'NoReg',
 'link for catchment SC #122': 'NoReg',
 'link for catchment SC #123': 'NoReg',
 'link for catchment SC #124': 'NoReg',
 'link for catchment SC #125': 'NoReg',
 'link for catchment SC #126': '120219A',
 'link for catchment SC #127': '120219A',
 'link for catchment SC #128': '120219A',
 'link for catchment SC #129': '120206A',
 'link for catchment SC #130': '120206A',
 'link for catchment SC #131': '120206A',
 'link for catchment SC #132': '120206A',
 'link for catchment SC #133': '120206A',
 'link for catchment SC #134': '120206A',
 'link for catchment SC #135': '120206A',
 'link for catch

In [23]:
kill_all_now(processes)